In [5]:
import os
from dotenv import load_dotenv

load_dotenv('.env')

test_var = os.getenv("TEST_VAR")
subscription_key = os.getenv("SUBSCRIPTION_KEY")
region = os.getenv("REGION")


language = "en-US"  # Example: 'en-US' for English, 'hi-IN' for Hindi


In [6]:
import pandas as pd
from tqdm.auto import tqdm

In [7]:
# USING MICROPHONE

import os
import azure.cognitiveservices.speech as speechsdk

def recognize_from_microphone():
    speech_config = speechsdk.SpeechConfig(subscription=subscription_key, region=region)
    speech_config.speech_recognition_language=language

    audio_config = speechsdk.audio.AudioConfig(use_default_microphone=True)
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    print("Speak into your microphone.")
    speech_recognition_result = speech_recognizer.recognize_once_async().get()

    if speech_recognition_result.reason == speechsdk.ResultReason.RecognizedSpeech:
        print("Recognized: {}".format(speech_recognition_result.text))
    elif speech_recognition_result.reason == speechsdk.ResultReason.NoMatch:
        print("No speech could be recognized: {}".format(speech_recognition_result.no_match_details))
    elif speech_recognition_result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = speech_recognition_result.cancellation_details
        print("Speech Recognition canceled: {}".format(cancellation_details.reason))
        if cancellation_details.reason == speechsdk.CancellationReason.Error:
            print("Error details: {}".format(cancellation_details.error_details))
            print("Did you set the speech resource key and region values?")

recognize_from_microphone()

Speak into your microphone.
Recognized: Hello, how are you doing?


In [2]:
import azure.cognitiveservices.speech as speechsdk

def transcribe_audio_to_text(audio_file_path, subscription_key, region, language):
    speech_config = speechsdk.SpeechConfig(subscription=subscription_key, region=region)
    speech_config.speech_recognition_language = language
    audio_config = speechsdk.audio.AudioConfig(filename=audio_file_path)
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)
    result = speech_recognizer.recognize_once()

    if result.reason == speechsdk.ResultReason.RecognizedSpeech:
        return result.text
    elif result.reason == speechsdk.ResultReason.NoMatch:
        return "No speech could be recognized"
    elif result.reason == speechsdk.ResultReason.Canceled:
        return f"Speech Recognition canceled: {result.cancellation_details.reason}, {result.cancellation_details.error_details}"

In [3]:
ROOT_DIR = "./data/audios/test/"
files = os.listdir(ROOT_DIR)
# files

In [ ]:
audio_file_path = [ROOT_DIR + file for file in files if file.endswith(".wav")][0]

# Transcribe the audio file and print the result
print(transcribe_audio_to_text(audio_file_path, subscription_key, region, language))


In [4]:
df = pd.read_csv('data/audios/recordings_info.csv')
df.head()

,file_name,phrase,prompt
0,1249120_43453425_58166571.wav,When I remember her I feel down,Emotional pain
1,1249120_43719934_43347848.wav,When I carry heavy things I feel like breaking...,Hair falling out
2,1249120_43719934_53187202.wav,there is too much pain when i move my arm,Heart hurts
3,1249120_31349958_55816195.wav,My son had his lip pierced and it is swollen a...,Infected wound
4,1249120_43719934_82524191.wav,My muscles in my lower back are aching,Infected wound


In [5]:
# main_df = pd.DataFrame(columns=['file', 'og_transcription', 'ms_asr_transcription'])

# print('1249120_1853182_11719913.wav' in df['file_name'].tolist())

In [ ]:
main_df = []

for i, data in tqdm(enumerate(zip(df['file_name'], df['phrase'])), total=len(df)):
    file, transcription = data[0], data[1]
    if file in files:
        file = ROOT_DIR + file
        # print(file)
        asr_transcription = transcribe_audio_to_text(file, subscription_key, region, language)
        main_df.append([file, transcription, asr_transcription])
    
    if i%10 ==0:
        m_df = pd.DataFrame(main_df, columns=['file', 'og_transcription', 'ms_asr_transcript/ion'])
        m_df.to_csv('main_df_test.csv', index=False)

In [ ]:
main_df = pd.DataFrame(main_df, columns=['file', 'og_transcription', 'ms_asr_transcript/ion'])
main_df.head()

,file,og_transcription,ms_asr_transcription
0,file,og_transcription,ms_asr_transcription
1,audios/1249120_44197979_23991689.wav,I have a sharp pain in my lower stomach.,I have a shopping in my lowest.
2,audios/1249120_44174241_12756709.wav,Don't cry,Don't try.
3,audios/1249120_44197979_109954671.wav,When I get out of bed in the morning my body f...,"When I get out of bed in the morning, my body ..."
4,audios/1249120_44246595_34981278.wav,i have a great pain in my thorax from heart in...,I have a great pain in my thorax from heart in...


In [74]:
main_df.to_csv('testing.csv', index=False)